# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [2]:
def movie_title_url_actor():
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    movie = soup.select("div.lst_wrap li")
    total_dic = []
    for i, m in enumerate(movie):
        if i >  4: # 상위 5개 영화만 출력
            break
        title = m.select_one("dt.tit a") #영화 제목은 df 의 title class의 a tag에 있음
        actors = m.select("dl.info_txt1 dd:nth-of-type(3) a") #배우들은 3번째 dd의 a tag에 있음
        movie_dic = {}
        
        movie_tit = str(title.text)
        movie_dic['title'] = movie_tit #title key에 영화제목 저장
        
        total_actors = []
        for idx, a in enumerate(actors):
            if idx > 2:
                break        
            #print(a.text)
            total_actors.append(a.text) #영화배우 3명을 list에 저장
        movie_dic['actors'] = total_actors #actors key에 영화배우list를 저장
        
        total_dic.append(movie_dic) #title과 actors가 입력 된 dictionary를 list에 저장
    return total_dic #list를 반환

In [3]:
movie_title_url_actor()

[{'title': '지푸라기라도 잡고 싶은 짐승들', 'actors': ['전도연', '정우성', '배성우']},
 {'title': '인비저블맨', 'actors': ['엘리자베스 모스']},
 {'title': '1917', 'actors': ['조지 맥케이', '딘-찰스 채프먼']},
 {'title': '정직한 후보', 'actors': ['라미란', '김무열', '나문희']},
 {'title': '작은 아씨들', 'actors': ['시얼샤 로넌', '엠마 왓슨', '플로렌스 퓨']}]

### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [4]:
def get_grade():
    """
    네티즌 평점, 관람객 평점, 기자/평론가 평점은 제목을 클릭하고 들어가야지 볼 수 있다
    제목을 클릭한 후 url을 살펴보면 
    url = 'https://movie.naver.com/ + /movie/bi/mi/basic.nhn?code = ?? 형식이다
    code가 포함된 url 형식은 dt의 tit class의 a tage에 있다.
    """
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    total_dic = [] #관람객, 기자/평론가, 네티즌
    
    movielinks = soup.select('dt.tit  a[href]')
    
    for idx, movielink in enumerate(movielinks): 
        if idx > 4 :
            break
        link = str(movielink.get('href')) 

        url2 = 'https://movie.naver.com' + link #영화 제목으로 들어가는 새로운 url
        movie_dic = {}
        res_2 = requests.get(url2)
        html_2 = res_2.text
        soup_2 = BeautifulSoup(html_2, 'html.parser')
        soup_2.select('div[class=star_score]')
        
        
        star = re.compile('[0-9]{2}.[0-9]{1}')# X.XX 형식의 정규표현식을 찾아야징~
        """
        별점 저장이 XX.X으로 되어있다.
        select('div[class=star_score]')[i].select("span.st_on")[0].get('style'))[i]로 별점을 받아와 0.1을 곱해주고,
        소숫점을 뒤에 2자리 까지만 표시한다.
        
        별점이 없는 경우에는 None을 입력한다.
        """
        for i in range(3):
            if i == 0 : # i = 0 :관람객 평점
                try: 
                    movie_dic['star_v'] = round(float(star.findall(soup_2.select('div[class=star_score]')[i].select("span.st_on")[0].get('style'))[0])*0.1,2)
                except : 
                     movie_dic['star_v'] = "None"
            elif i == 1 :#i = 1 :기자/평론가 평점
                try: 
                    movie_dic['star_j'] = round(float(star.findall(soup_2.select('div[class=star_score]')[i].select("span.st_on")[0].get('style'))[0])*0.1,2)
                except : 
                     movie_dic['star_j'] = "None"
            
            else : #i = 2 :네티즌 평점
                try :
                     movie_dic['star_n'] = round(float(star.findall(soup_2.select('div[class=star_score]')[i].select("span.st_on")[0].get('style'))[0])*0.1,2)
                except :
                     movie_dic['star_n'] = "None"
        
        total_dic.append(movie_dic)
           
    return total_dic

In [5]:
get_grade()

[{'star_v': 8.33, 'star_j': 6.71, 'star_n': 6.79},
 {'star_v': 7.75, 'star_j': 8.06, 'star_n': 'None'},
 {'star_v': 9.3, 'star_j': 7.67, 'star_n': 8.98},
 {'star_v': 8.6, 'star_j': 5.38, 'star_n': 7.7},
 {'star_v': 9.18, 'star_j': 8.0, 'star_n': 8.88}]

### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [12]:
def get_reviews():
    
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')


    movielinks = soup.select('dt.tit  a[href]')
    total_dic = []
    
    for idx, movielink in enumerate(movielinks):
        if idx > 4 :
            break
        movie_dic = {}
        comment = []#20개의 댓글을 저장 할 list
        star = [] #20개의 별점을 저장 할 list
        link = str(movielink.get('href')) 
        code = ''.join(re.findall("\d", link))#원래의 url에서 code뒤에 숫자만 필요하다. 숫자만 찾아서 str로 변환
        
        #첫번째 페이지에서 10개의 별점과 댓글을 crawling
        url2 = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + code
        res_2 = requests.get(url2)
        html_2 = res_2.text
        soup_2 = BeautifulSoup(html_2, 'html.parser')
    
        
        rep = soup_2.find('div', {'class': 'score_result'})
        try : 
            data = rep.find_all('li')
            for i in range(len(data)):
                star.append(soup_2.select("div.star_score em")[i].text)
                #별점은 div의 score result class안에 star score class안에 em tag에 있고, text만 추출한다.
                comment.append(data[i].find('div', {'class': 'score_reple'}).find('p').text[26:].strip()) 
                #댓글은 div의 score_result class안에 score_reple class엔에 p tag에 있다.
                # 앞에 불필요한 '관람객','스포일러가 포함된 댓글입니다.댓글보기'를 제외하고 append
            
        except :
            pass
            
        # 두번째 page에서 나머지 10개의 댓글과 별점 crawling. url2에 "&page=2"를 추가하면 2page로 넘어감  
        url2_2 = url2 + "&page=2"
        res_2_2 = requests.get(url2_2)
        html_2_2 = res_2_2.text
        soup_2_2 = BeautifulSoup(html_2_2, 'html.parser')
    
        
        rep = soup_2_2.find('div', {'class': 'score_result'})
        try : 
            data = rep.find_all('li')
            for i in range(len(data)):
                star.append(soup_2.select("div.star_score em")[i].text)
                #별점은 div의 score result class안에 star score class안에 em tag에 있고, text만 추출한다.
                comment.append(data[i].find('div', {'class': 'score_reple'}).find('p').text[26:].strip()) 
                #댓글은 div의 score_result class안에 score_reple class엔에 p tag에 있다.
                # 앞에 불필요한 '관람객','스포일러가 포함된 댓글입니다.댓글보기'를 제외하고 append
            
        except :
            pass
        movie_dic['comment_20'] = comment #댓글을 dictionary로 저장
        movie_dic['star_20'] = star #별점을 dictionary로 저장
        total_dic.append(movie_dic) #dictionary를 lsit에 저장
    return total_dic
   

In [13]:
get_reviews()

[{'comment_20': ['난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 지푸라기',
   '전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...',
   '8명의 배우가 모두 주인공 같은 느낌.',
   '개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ',
   '연출, 연기, 스토리 모두 대박...무조건 보세요.',
   '연기오지고 스릴오지고',
   '스토리가 짱짱하네요~ 심리적인 긴장감을 잘 살린 영화인것 같네요~ 인기좀 끌듯...',
   '한국식 피칠갑을 한 타란티노 영화',
   '연기 쩐다잉 ,,, 또 보고 싶음 ㅠ',
   '진짜 보고싶었던 영환데 드디어 봤습니당 기다린 보람이 있네용ㅋㅋㅋ 등장인물이 많았는데 영화 속에서 잘 풀어낸 것 같아요 강추합니당 !!',
   '아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽부터 함? 백번양보해서 시사회때 봤다 쳐도 이렇게나 많이 봤다고? 죄다 똑같은 말투에? 음원이고 영화고 조작질 역겹다 진짜',
   '전도연을 위한, 전도연에 의한 영화! 데뷔작이라고는 믿을수 없는 연출력에놀랐다~',
   '전도연 등장하자마자 걍 스크린 장악함ㅋㅋㅋㅋ역시 전도연이 선택한 작품은 안보고 넘어갈 수 없지',
   '이 캐스팅 너무 마음에 든다.영화보고나서도 할말이 많아지는 영화',
   '솔직히 이 영화 돈주고 볼만합니다ㅎㅎ',
   '믿고보는 칸의 여왕 전도연!! 정우성, 배성우 등등 진짜 미친 명품 조연들...!! 스릴러의 긴장도, 몰입도 괜찮은 영화~',
   '진짜 전도연은 대체불가다.',
   '다들너무연기를잘하고일단 이런 스토리탄탄한영화 오랜만이네요 굿굿구성도재밋고',
   '갖은 재료 다 집어넣었는데 왜 맛이 안나지?',
   '영화 보는 내내 제발 한순간만이라도 재미있는 장면 나오길 지푸라기 잡는 심정으로 봤는데 없음'],
  'star_20': ['10',
   '10',
   '10',
  

### 4. 저장하기

In [14]:
import json
def save(dictionary, filename):#dictionary를 json으로 저장해볼까나~
    with open(filename,'w') as fp:
        json.dump(dictionary, fp,sort_keys=True, indent=4,ensure_ascii=False)
        #한글도 읽을 수 있게 dump!

In [21]:
#dictionary마다 다른 jsion파일로 저장하려면
"""
import json
def save(dictionary, filename):
    for i in range(5):
        with open(filename + i,'w') as fp:
            json.dump(dictionary[i], fp,sort_keys=True, indent=4,ensure_ascii=False)
            #한글도 읽을 수 있게 dump!
"""

"\nimport json\ndef save(dictionary, filename):\n    for i in range(5):\n        with open(filename + i,'w') as fp:\n            json.dump(dictionary[i], fp,sort_keys=True, indent=4,ensure_ascii=False)\n            #한글도 읽을 수 있게 dump!\n"

### 5. 크롤링하기

In [15]:
data_t_a = movie_title_url_actor()

In [16]:
data_g = get_grade()

In [17]:
data_r = get_reviews()

In [18]:
for i in range(5):
    data_t_a[i].update(data_g[i])
    data_t_a[i].update(data_r[i])
#3개의 dictionary를 모두 합침
data = data_t_a

In [19]:
save(data,'SooA_S2.json')

총 5개의 dictionary를 json파일로 저장
(수아맘속에 저~장 ★ @v<)